In [16]:
import os
import pandas as pd
import numpy as np
import datetime
from datetime import datetime

data_dir = os.getcwd().replace("/notebooks","/data/interim")
csv_name = "combined_accounts.csv"
accts_path = os.path.join(data_dir, csv_name)

In [17]:
accts_df = pd.read_csv(accts_path)
accts_df.head()

,Source,date,Description,credit_flg,debit_flg,Amount,meta_category,category,sub_category,event
0,USAA CC,10/27/18,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-50.72,Discretionary,Alcohol,NaN,NaN
1,USAA CC,9/29/18,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-48.56,Discretionary,Alcohol,NaN,NaN
2,USAA CC,9/22/18,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-61.10,Discretionary,Alcohol,NaN,NaN
3,USAA CC,9/10/18,HIGHLAND WINE AND LIQUOR DENVER CO,False,True,-11.87,Discretionary,Alcohol,NaN,NaN
4,USAA CC,9/1/18,TOWN LIQUOR 0848700216503DENVER CO,False,True,-22.68,Discretionary,Alcohol,NaN,NaN


In [18]:
accts_df["date"] = pd.to_datetime(accts_df["date"])
accts_df["year"] = accts_df.apply(lambda x : x["date"].year, axis=1)
accts_df["month"] = accts_df.apply(lambda x : x["date"].month, axis=1)
accts_df["day"] = accts_df.apply(lambda x : x["date"].day, axis=1)

In [19]:
accts_df.head()

,Source,date,Description,credit_flg,debit_flg,Amount,meta_category,category,sub_category,event,year,month,day
0,USAA CC,2018-10-27,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-50.72,Discretionary,Alcohol,NaN,NaN,2018,10,27
1,USAA CC,2018-09-29,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-48.56,Discretionary,Alcohol,NaN,NaN,2018,9,29
2,USAA CC,2018-09-22,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-61.10,Discretionary,Alcohol,NaN,NaN,2018,9,22
3,USAA CC,2018-09-10,HIGHLAND WINE AND LIQUOR DENVER CO,False,True,-11.87,Discretionary,Alcohol,NaN,NaN,2018,9,10
4,USAA CC,2018-09-01,TOWN LIQUOR 0848700216503DENVER CO,False,True,-22.68,Discretionary,Alcohol,NaN,NaN,2018,9,1


In [20]:
accts_df.meta_category.unique()

array(['Discretionary', 'Hobby', 'Experiences', 'Avoidable', 'Required',
       'Transportation', 'Personal Care', 'Debt Payment', 'Dating',
       'Health', 'Entertainment', 'Grocery', 'Savings', 'Miscellaneous',
       'Tiny Home', nan], dtype=object)

In [21]:
debits_df = accts_df[(accts_df.debit_flg == True) & (accts_df.year == 2018)]
debits_df = debits_df[debits_df.category != "Credit Card Payment"]
#debits_df = debits_df[debits_df.meta_category != "Savings"]
groupby_df = debits_df.groupby(['month', 'meta_category'])['Amount'].sum().reset_index()
groupby_df

,month,meta_category,Amount
0,5,Discretionary,-1211.32
1,5,Entertainment,-47.95
2,5,Experiences,-854.35
3,5,Grocery,-433.88
4,5,Health,-140.00
5,5,Hobby,-363.12
6,5,Personal Care,-48.40
7,5,Required,-1552.81
8,5,Savings,-300.00
9,5,Transportation,-518.47


In [22]:
debits_df[debits_df.month == 7].to_csv("/Users/gbolla/Desktop/july.csv")
groupby_df.to_csv("/Users/gbolla/Desktop/category_spending.csv")

In [39]:
credits_df = accts_df[accts_df.credit_flg == True]
credits_df[credits_df.month == 10].to_csv("/Users/greg.bolla/Desktop/october_credits.csv")